In [ ]:
#Extra

#1 - Export environment variables

In [ ]:
#Main

#Import


In [295]:
#data_completer.py

def data_completer(pair, interval, start_time, max_requests):
    
    table_name = 'binance_klines_' + pair + '_' + interval
    
    keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
       'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}
    
    max_attempts = 10
    
    #Atualiza o status do par em operação para 'building'
    update_table_job_status = update_table('binance_pairs', 'name', pair, 'status', 'building')
    #É preciso tratar a possível exceção update_table_job_status = 'fail'
    
    print('Iniciando complemento de dados para o par ',pair)
    print(' ')
    print('Atualizando para building: ', update_table_job_status)
    print(' ')
    
    total_num_requests = 0
    round_counter = 1
    
    while True:
        
        print('ciclo ', round_counter)
        klines = []
        num_requests_each_round = 0
        
        klines, num_requests_each_round = binance_klines_request_handler(pair, interval, start_time, max_attempts)
        
        total_num_requests = total_num_requests + num_requests_each_round
        
        if (len(klines) > 0):
            
            print('klines vieram!')
            
            if (len(klines) < 500): #Chegou na kline mais recente
                
                klines = klines[:(len(klines)-1)] #Apaga o último resgistro (candle não fechado)
            
            most_recent_open_time = klines[len(klines)-1][0] #open_time do último candle
            
            klines = binance_klines_to_postgres_klines(klines) #Parse das klines
            
            save_in_table_job_status = save_in_table(table_name, keys, klines)
            
            start_time = str(most_recent_open_time + 30000) #Um passo de 30s (30000 milissegundos)
            
            print ('Leva de ',len(klines),' salvas no banco. Total de requests até agora: ',total_num_requests)
            print(' ')
            
            if (len(klines) < 500):
                
                data_completer_job_status = 'done'
                
                print('kline mais recente')
                

                break

            if (len(klines) > 500): #Alguma anomalia.

                #print('ATENÇÃO, MAIS DE 500 AMOSTRAS')
                print('ANOMALIA')
                
                break
                
        else: #ERRO DE COMUNICAÇÃO COM A BINANCE descansa e trata log
            
            print('ERRO DE COMUNICAÇÃO COM A BINANCE')
        
        if(total_num_requests > int(max_requests)):

            print('número máximo de requests atingido, dormindo 1 min')
            print(' ')
            total_num_requests = 0 #zera o contador de requests
            time.sleep(60)
            
        round_counter+=1

    #Atualiza o status do par em operação para 'full'
    update_table_job_status = update_table('binance_pairs', 'name', pair, 'status', 'full')
    #É preciso tratar a possível exceção update_table_job_status = 'fail'
    
    print(' ')
    print('Atualizando para full: ', update_table_job_status)
    print(' ')
    print('FIM')



## Import

## 3 - data_completer

***Deslocar para local apropriado***

In [296]:
keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
       'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}

pair = 'ETHUSDT'

interval = '1m'

start_time = '1241893500000'

max_requests = 1000

table_name = 'binance_klines_' + pair + '_' + interval

#print(table_name)

create_table_job_satus = create_table(table_name,keys,pk='open_time')

In [297]:

data_completer(pair, interval, start_time, max_requests)

Iniciando complemento de dados para o par  ETHUSDT
 
Atualizando para building:  done
 
ciclo  1
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  1
 
ciclo  2
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  2
 
ciclo  3
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  3
 
ciclo  4
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  4
 
ciclo  5
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  5
 
ciclo  6
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  6
 
ciclo  7
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  7
 
ciclo  8
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  8
 
ciclo  9
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  9
 
ciclo  10
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  10
 
ciclo  11
klines vie

Leva de  500  salvas no banco. Total de requests até agora:  90
 
ciclo  91
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  91
 
ciclo  92
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  92
 
ciclo  93
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  93
 
ciclo  94
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  94
 
ciclo  95
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  95
 
ciclo  96
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  96
 
ciclo  97
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  97
 
ciclo  98
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  98
 
ciclo  99
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  99
 
ciclo  100
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  100
 
ciclo  101
klines vier

klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  179
 
ciclo  180
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  180
 
ciclo  181
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  181
 
ciclo  182
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  182
 
ciclo  183
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  183
 
ciclo  184
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  184
 
ciclo  185
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  185
 
ciclo  186
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  186
 
ciclo  187
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  187
 
ciclo  188
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  188
 
ciclo  189
klines vieram!
Leva de  500  salvas no banco. Total de requests até ag

Leva de  500  salvas no banco. Total de requests até agora:  267
 
ciclo  268
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  268
 
ciclo  269
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  269
 
ciclo  270
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  270
 
ciclo  271
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  271
 
ciclo  272
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  272
 
ciclo  273
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  273
 
ciclo  274
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  274
 
ciclo  275
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  275
 
ciclo  276
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  276
 
ciclo  277
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  277
 
cic

klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  356
 
ciclo  357
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  357
 
ciclo  358
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  358
 
ciclo  359
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  359
 
ciclo  360
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  360
 
ciclo  361
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  361
 
ciclo  362
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  362
 
ciclo  363
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  363
 
ciclo  364
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  364
 
ciclo  365
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  365
 
ciclo  366
klines vieram!
Leva de  500  salvas no banco. Total de requests até ag

...

Leva de  500  salvas no banco. Total de requests até agora:  500
 
ciclo  1502
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  501
 
ciclo  1503
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  502
 
ciclo  1504
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  503
 
ciclo  1505
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  504
 
ciclo  1506
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  505
 
ciclo  1507
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  506
 
ciclo  1508
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  507
 
ciclo  1509
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  508
 
ciclo  1510
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  509
 
ciclo  1511
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora: 

klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  588
 
ciclo  1590
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  589
 
ciclo  1591
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  590
 
ciclo  1592
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  591
 
ciclo  1593
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  592
 
ciclo  1594
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  593
 
ciclo  1595
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  594
 
ciclo  1596
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  595
 
ciclo  1597
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  596
 
ciclo  1598
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  597
 
ciclo  1599
klines vieram!
Leva de  500  salvas no banco. Total de reque

Leva de  500  salvas no banco. Total de requests até agora:  675
 
ciclo  1677
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  676
 
ciclo  1678
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  677
 
ciclo  1679
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  678
 
ciclo  1680
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  679
 
ciclo  1681
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  680
 
ciclo  1682
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  681
 
ciclo  1683
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  682
 
ciclo  1684
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  683
 
ciclo  1685
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  684
 
ciclo  1686
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora: 

klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  763
 
ciclo  1765
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  764
 
ciclo  1766
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  765
 
ciclo  1767
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  766
 
ciclo  1768
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  767
 
ciclo  1769
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  768
 
ciclo  1770
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  769
 
ciclo  1771
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  770
 
ciclo  1772
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  771
 
ciclo  1773
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  772
 
ciclo  1774
klines vieram!
Leva de  500  salvas no banco. Total de reque

Leva de  500  salvas no banco. Total de requests até agora:  850
 
ciclo  1852
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  851
 
ciclo  1853
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  852
 
ciclo  1854
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  853
 
ciclo  1855
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  854
 
ciclo  1856
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  855
 
ciclo  1857
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  856
 
ciclo  1858
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  857
 
ciclo  1859
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  858
 
ciclo  1860
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora:  859
 
ciclo  1861
klines vieram!
Leva de  500  salvas no banco. Total de requests até agora: 

In [287]:
#Db functions


#Import

import psycopg2
import psycopg2.extras
import os
from datetime import datetime
#from general_functions import log_handler

#Environment
'''
POSTGRES_DB = os.environ.get('POSTGRES_DB')
POSTGRES_USER = os.environ.get('POSTGRES_USER')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
DB_HOST = os.environ.get('DB_HOST')
'''
#APAGAR####################################
DB_HOST='localhost'
POSTGRES_DB='pg_dev' 
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'
POSTGRES_USER='openbot'
###########################################

#Functions

def create_table(table_name,keys,**kwargs):
    
    """Cria a tabela <table_name>, com as colunas <keys[i]>. Opcionalmente pode-se definir uma das chaves como 
    chave primária, ao invés da chave default atribuida pelo postgres.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser criada
    keys        -- Chaves (nomes das colunas)


    **kwargs:
    ========
    
    pk   -- Chave que será atribuida como primária 
    mute -- Suprime o log se 'yes'
    """
    
    pk = kwargs.get('pk')
    
    mute = kwargs.get('mute')
    
    connection = '' #Inicialmente definida como string
    
    records = [] #Inicialmente definida como uma lista vazia
    
    create_table_job_status = 'fail' 
    
    default_log = 'print'
    
    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    #Não conecta ao banco
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

        msg = '''
FROM: create_table
''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''.

'''
        
        log_handler(msg,default_log)

    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados    
    
            #Tenta criar a tabela
            try:
                
                sql_create_query = sql_command(table_name, keys, 'create')
                
                if (pk):
                    
                    sql_create_query = sql_command(table_name, keys, 'create', pk=pk)
                
                pointer.execute(sql_create_query)
                
                connection.commit()
                
                create_table_job_status = 'done'

            #Erro aocriar tabela
            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
                
                #Desfaz ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM: create_table
''' + str(datetime.utcnow()) + '''

Não foi possível criar tabela ''' + table_name + ''' devido ao erro: 
''' + str(error) + '''.

'''
   
                log_handler(msg,default_log)
    
    return create_table_job_status

                    
def read_table(table_name, **kwargs):
    
    """Busca no banco a tabela [table_name], retornando uma lista, tal que cada elemento representa
    uma linha (registro) da tabela, ou uma lista vazia, caso a tabela não seja encontrada. Pode retornar
    a tabela inteira, ou parte dos valores se forem informados, corretamente, os argumentos opcionais
    **kwargs.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser buscada
    
    **kwargs:
    =========
    
    mute -- Suprime o log se 'yes'
    
    field_key -- Chave (nome da coluna) do campo tomado como referência de ordenamento
    
    sort_type      -- Modo como os registros são buscados:
        
    *sort_type options*
     -----------------
        - DESC ==> Do maior para o menor
        - ASC  ==> Do menor para o maior
        
    limit     -- Número de registros que devem ser retornados
    """
    
    field_key = kwargs.get('field_key')
    
    sort_type = kwargs.get('sort_type')
    
    limit = kwargs.get('limit')

    mute = kwargs.get('mute')
    
    connection = '' #Inicialmente definida como string
    
    records = [] #Inicialmente definida como uma lista vazia
    
    default_log = 'print'

    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
        
        msg = '''
FROM: read_table
''' + str(datetime.utcnow()) + '''

Conectado ao banco com sucesso!

'''

        log_handler(msg,default_log)

    #Não conecta ao banco    
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

        msg = '''
FROM: read_table
''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''

'''
        
        log_handler(msg,default_log)

    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados
            
            #Tenta obter a tabela
            try:
                
                sql_select_query = 'SELECT * FROM ' + table_name
                
                if (field_key and sort_type and limit):
                    
                    sql_select_query = sql_select_query + ' ORDER BY ' + field_key + ' ' + sort_type + ' LIMIT ' + limit

                pointer.execute(sql_select_query)

                records = pointer.fetchall()
                
                msg = '''
FROM: read_table
''' + str(datetime.utcnow()) + '''

Tabela ''' + table_name +''' obtida com sucesso! 

'''
                
                log_handler(msg,default_log)

            #Não encontra tabela
            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
                
                #Desfaz ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM: read_table
''' + str(datetime.utcnow()) + '''

Não foi possível obter tabela ''' + table_name +''' devido ao erro: 
''' + str(error) + '''

'''
                
                log_handler(msg,default_log)
            
            pointer.close()
            connection.close()
    
    return records


def update_table(table_name, pk_field, pk_value, field_to_update, new_field_value, **kwargs):
  
    """Dada uma certa tabela, atualiza uma entrada específica da mesma.
    
    
    Keyword arguments:
    =================
    
    table_name      -- Nome da tabela a ser buscada
    pk_field        -- Chave (nome da coluna) atribuída como chave primária; ``pk``, se chave default.
    pk_value        -- Valor da chave primária do registro
    field_to_update -- Chave (nome da coluna) do campo a ser regravado
    new_field_value -- Novo valor a ser gravado no campo
    
    **kwargs:
    =========
    
    mute -- Suprime o log se 'yes'
    """
    
    mute = kwargs.get('mute')

    connection = ''
    
    update_table_job_satus = 'fail'
    
    default_log = 'print'
    
    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    #Não conecta ao banco
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
        
        msg = '''
FROM: update_table
''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''

'''
        log_handler(msg,default_log)
        
    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados
            
            #Tenta atualizar entrada na tabela
            try:

                sql_update_query = 'Update ' + table_name + ' set ' + field_to_update + ' = %s where ' + pk_field + '= %s'
                
                pointer.execute(sql_update_query, (new_field_value, pk_value))

                connection.commit()

                update_table_job_satus = 'done'

            #Não consegue atualizar entrada
            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

                #Desfaz a ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM: update_table
''' + str(datetime.utcnow()) + '''

Tabela ''' + table_name + ''': Não foi possível atualizar a entrada ''' + field_to_update + ''' 
do registro ''' + pk_value + ''' para ''' + new_field_value + ''' devido ao erro: 
''' + str(error) + '''

'''
   
                log_handler(msg,default_log)
    
            pointer.close()
            connection.close()

    return update_table_job_satus


def save_in_table(table_name, keys, data, **kwargs):
    
    """Salva na tabela <table_name>, nas respectivas colunas <keys[i]>, os dados correspondentes, <data>.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela na qual os dados devem ser gravados
    keys        -- Chaves (nomes das colunas)
    data_list   -- Lista de dados a serem gravados


    **kwargs:
    ========

    mute -- Suprime o log se 'yes'
    """

    mute = kwargs.get('mute')

    connection = ''
    
    save_in_table_job_satus = 'fail'
    
    default_log = 'print'
    
    if(mute):
        
        if(mute == 'yes'): default_log = 'mute'
    
    #Tenta conectar ao banco
    try:

        connection = psycopg2.connect(host=DB_HOST, database=POSTGRES_DB, 
                                      user=POSTGRES_USER, password=POSTGRES_PASSWORD)

        pointer = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    #Não conecta ao banco
    except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI
        
        msg = '''
FROM: save_in_table
''' + str(datetime.utcnow()) + '''

Não foi possível conectar ao banco devido ao erro: 
''' + str(error) + '''.

'''
        
        log_handler(msg,default_log)
        
    finally:

        if (type(connection) != str): #Conexão bem sucedida com o banco de dados
            

            #Tenta salvar os dados na tabela
            try:
                
                sql_save_query = sql_command(table_name, keys, 'save')
                
                for i in range(len(data)):
                    
                    pointer.execute(sql_save_query, data[i])
                
                connection.commit()
                
                save_in_table_job_satus = 'done'

            except (Exception, psycopg2.Error) as error: #TRATAR EXCEÇÃO AQUI

                #Desfaz ação do ponteiro
                pointer.execute("ROLLBACK")

                msg = '''
FROM: save_in_table
''' + str(datetime.utcnow()) + '''

Não foi possível salvar dados na tabela: ''' + table_name + ''' devido ao erro: 
''' + str(error) + '''.

'''

                log_handler(msg,default_log)
    
    
    return save_in_table_job_satus

In [201]:
#binance functions

#Import
import requests
import json
import time

#ESCREVER DOCSTRING
def binance_klines_request_handler(pair, interval, start_time, max_attempts):
    
    num_requests = 0
    
    klines = []
    
    url = '''https://api.binance.com/api/v1/klines?symbol=''' + pair + '''&interval=''' + interval + '''&startTime=''' + start_time
    
    default_log = 'print'
    
    for i in range(max_attempts):
        
        #Tenta conexão com a Binance
        num_requests+= 1
        try:

            response = requests.get(url)
            
            response.raise_for_status()
                        
        #Request falhou
        except (Exception, requests.exceptions.RequestException) as error: #TRATAR EXCEÇÃO AQUI
            
            msg = '''
FROM: get_historical_data
''' + str(datetime.utcnow()) + '''

Par ''' + pair + ''': Comunicação com a Binance falhou no ''' + str(i+1) + '''\u00bA request, devido ao erro:

''' + str(error) + '''

'''
            
            log_handler(msg,default_log)
            
            time.sleep(5)
            
        finally:
            
            if (int(response.status_code) == 200):
                
                klines = response.json()
                
                break

    return klines, num_requests

In [285]:
#Other functions

#Import

import sys
import os
from datetime import datetime
from modules import notification

def log_handler(msg,destination):
    
    """Envia uma mensagem de log para o destino escolhido pelo usuário.
    
    
    Keyword arguments:
    =================
    
    msg         -- Mensagem de log
    destination -- Caminho padrão para saída da mensagem de log
    
    *destination options*
     -------------------
        - mute   ==> Ignora a mensagem
        - print  ==> Printa na tela
        - logm   ==> Envia para o arquivo padrão de monitoramento
        - logw   ==> Envia para o arquivo padrão de alertas
        - tlgrm  ==> Envia para o grupo de monitoramento do telegram
        - tlgrw  ==> Envia para o grupo de alertas do telegram    
    """
    
    if (destination == 'mute'):
        
        pass
    
    elif (destination == 'print'):
        
        print(msg)
    
    elif (destination == 'logm'):
        
        path_to_logfile = os.getcwd().split(sep="BinanceDataStorageDaemon/")[0] + 'logs/binance_data_monitoring.log'
        sys.stdout = open(path_to_logfile, "a")
        print(msg)
        
    elif (destination == 'logw'):
        
        path_to_logfile = os.getcwd().split(sep="BinanceDataStorageDaemon/")[0] + 'logs/binance_data_warning.log'
        sys.stdout = open(path_to_logfile, "a")
        print(msg)
        
    elif (destination == 'tlgrm'):
        
        notification.Telegrambot(msg).monitoring()
        
    elif (destination == 'tlgrw'):
        
        notification.Telegrambot(msg).warning()


def sql_command(table_name, keys, action, **kwargs):
    
    """Dado um nome de tabela <table_name> e o dicionário <keys> (em que a chave será o nome da coluna e 
    o valor definirá o tipo de dado gravado - vide documentação do PostgreSQL), retorna o comando SQL ade-
    quado para criação da tabela ou salvamento de dados em uma tabela.
    
    
    Keyword arguments:
    =================
    
    table_name  -- Nome da tabela a ser buscada
    keys        -- Dicionário com as chaves e seus respectivos tipos
    action      -- O tipo de ação desejada com o comando SQL, criar ou salvar.

    *action options*
     ------------
        - create ==> SQL para criar uma tabela
        - save   ==> SQL para salvar em uma tabela
    
    **kwargs:
    =========
    
    pk -- Chave primária
    """
    
    pk = kwargs.get('pk')
    
    key_list = list(keys.keys())
    type_list = list(keys.values())
    
    sql = ''
    
    if (action == 'create'):
        
        sql = 'CREATE TABLE ' + table_name + ' ('
        
        for i in range (len(keys)-1):
            
            entry = key_list[i] + ' ' + type_list[i]
            
            if (pk):
                
                if (key_list[i] == pk):
                    
                    entry = key_list[i] + ' ' + type_list[i] + ' PRIMARY KEY'
                        
            sql = sql + entry + ', '

        sql = sql + key_list[len(keys)-1] + ' ' + type_list[len(keys)-1] + ')'
        
    if (action == 'save'):
        
        format_code = ''
        
        sql = 'INSERT INTO ' + table_name + ' ('
        
        for i in range (len(keys)-1):
            
            sql = sql + key_list[i] + ' ' + ', '
            
            format_code = format_code + '%s, '

        sql = sql + key_list[len(keys)-1] + ' ' + ') VALUES (' + format_code + '%s)'
                        
    return sql

#ESCREVER DOCSTRING
def binance_klines_to_postgres_klines(in_dt):
    
    out_dt = []
    
    for i in range (len(in_dt)):
        
        data = (datetime.fromtimestamp(in_dt[i][0]/1000), in_dt[i][1], in_dt[i][2], in_dt[i][3], 
                     in_dt[i][4], in_dt[i][5], in_dt[i][7], in_dt[i][8], in_dt[i][9], in_dt[i][10])
    
        out_dt.append(data)
    
    return out_dt

# Zona de testes

## 1 - Funções banco de dados

### 1.1 - create_table

In [ ]:
#=======================================================================
#Função auxiliar para gerar uma string randômica
import random
import string

def randomString(stringLength=10):
    
    """Generate a random string of fixed length """
    
    letters = string.ascii_lowercase
    
    return ''.join(random.choice(letters) for i in range(stringLength))
#=======================================================================

recent_created_table_name = 'test_table_' + randomString(10)

keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
       'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}

create_table_job_satus = create_table(recent_created_table_name,keys,pk='open_time')

print(create_table_job_satus)

### 1.2 - read_table

In [114]:
#Coletando todas as amostras

table_name = 'binance_pairs'

records = read_table(table_name)

if (len(records) > 0):
    
    for record in records:
        
        print (record)


FROM: read_table
2019-06-09 00:15:12.830007

Conectado ao banco com sucesso!



FROM: read_table
2019-06-09 00:15:12.831050

Tabela binance_pairs obtida com sucesso! 


['ETHUSDT', 'OFF', 'absent', None, None]
['BNBUSDT', 'OFF', 'absent', None, None]
['ADAUSDT', 'OFF', 'absent', None, None]
['XRPUSDT', 'OFF', 'absent', None, None]
['BTCUSDT', 'OFF', 'absent', None, None]


In [113]:
#Coletando uma parte das amostras

table_name = 'nnsphwirsj'

records = read_table(table_name, field_key = 'open_time', sort_type = 'DESC', limit = '10')

if (len(records) > 0):
    
    print('''open_time | open | high | low | close | volume | quote_asset_volume | number_of_trades | 
          taker_buy_base_asset_volume | taker_buy_quote_asset_volume | 
          ''')
    
    for record in records:
        
        print (record['open_time'], record['open'], record['high'], record['low'], record['close'], 
               record['volume'], record['quote_asset_volume'], record['number_of_trades'], record['taker_buy_base_asset_volume'], record['taker_buy_quote_asset_volume'])
        print(' ')


FROM: read_table
2019-06-09 00:11:48.355769

Conectado ao banco com sucesso!



FROM: read_table
2019-06-09 00:11:48.358355

Tabela nnsphwirsj obtida com sucesso! 


open_time | open | high | low | close | volume | quote_asset_volume | number_of_trades | 
          taker_buy_base_asset_volume | taker_buy_quote_asset_volume | 
          
2017-08-17 09:19:00 4431.68000000 4431.68000000 4431.68000000 4431.68000000 0.02500000 110.79200000 1 0E-8 0E-8
 
2017-08-17 09:18:00 4441.87000000 4441.87000000 4441.87000000 4441.87000000 0E-8 0E-8 0 0E-8 0E-8
 
2017-08-17 09:17:00 4441.87000000 4441.87000000 4441.87000000 4441.87000000 0.25670200 1140.23691274 3 0.25670200 1140.23691274
 
2017-08-17 09:16:00 4441.87000000 4441.87000000 4441.87000000 4441.87000000 0.06616300 293.88744481 1 0.06616300 293.88744481
 
2017-08-17 09:15:00 4441.87000000 4441.87000000 4441.87000000 4441.87000000 0.08199800 364.22445626 1 0.08199800 364.22445626
 
2017-08-17 09:14:00 4441.87000000 4441.87000000 4441.8700000

### 1.3 - update_table

In [19]:
#Testando para mudança da chave 'status', da tabela 'binance_pairs' Modifique À vontade.

#=======================================================================
#Função auxiliar para gerar uma string randômica
import random
import string

def randomString(stringLength=10):
    
    """Generate a random string of fixed length """
    
    letters = string.ascii_lowercase
    
    return ''.join(random.choice(letters) for i in range(stringLength))
#=======================================================================


table_name = 'binance_pairs'

pk_field = 'name'

pk_value = 'BTCUSDT'

field_to_update = 'status'

new_field_value = randomString(8) #Máximo 8 caracteres

records = read_table(table_name, mute='yes')

for record in records:

    if (record[pk_field] == pk_value):

        old_record = record
        
        old_field_value = record[field_to_update]

update_table_job_status = update_table(table_name, pk_field, pk_value, field_to_update, new_field_value)

print(update_table_job_status + '''
''')

if (update_table_job_status == 'done'):
    
    records = read_table(table_name, mute='yes')
    
    for record in records:
        
        if (record[pk_field] == pk_value):
            
            new_record = record
            
    #Desfazendo
    update_table_job_status = update_table(table_name, pk_field, pk_value, field_to_update, old_field_value)
    
print('''Entrada anterior   - ''' + str(old_record) + '''
Entrada atualizada - ''' + str(new_record) + '''
''')

done

Entrada anterior   - ['BTCUSDT', 'OFF', 'absent', None, None]
Entrada atualizada - ['BTCUSDT', 'OFF', 'vzelynbe', None, None]



### 1.4 - save_in_table


In [286]:
#=======================================================================
#Função auxiliar para gerar uma string randômica
import random
import string

def randomString(stringLength=10):
    
    """Generate a random string of fixed length """
    
    letters = string.ascii_lowercase
    
    return ''.join(random.choice(letters) for i in range(stringLength))
#=======================================================================

recent_created_table_name = 'test_table_' + randomString(10)

keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
        'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
       'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}

create_table_job_satus = create_table(recent_created_table_name,keys,pk='open_time',mute='yes')

if (create_table_job_satus == 'done'):
    
    print ('Nova tabela criada com o nome: ',recent_created_table_name)

    import requests
    import json

    url = 'https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1m&startTime=1241893500000'

    response = requests.get(url)

    klines = response.json()

    #for kline in klines:

    klines = binance_klines_to_postgres_klines(klines)

    save_in_table_job_status = save_in_table(recent_created_table_name, keys, klines)

    print ('save_in_table_job_status = ',save_in_table_job_status)

Nova tabela criada com o nome:  ccmalepzek
save_in_table_job_status =  done


## 2 - Funções da binance

### 2.1 - binance_klines_request_handler

In [213]:
pair = 'ETHUSDT'
interval = '1m'
start_time = '1241893500000'
max_attempts = 10

klines, num_requests = binance_klines_request_handler(pair, interval, start_time, max_attempts)

print('O total de requests foi: ', num_requests)
print(' ')


if (len(klines) > 0):
    
    
    for i in range (5):
        
        print(klines[i] , '''
        ''')

O total de requests foi:  1
 
[1502942400000, '301.13000000', '301.13000000', '301.13000000', '301.13000000', '0.42643000', 1502942459999, '128.41086590', 2, '0.42643000', '128.41086590', '46530.51064628'] 
        
[1502942460000, '301.13000000', '301.13000000', '301.13000000', '301.13000000', '2.75787000', 1502942519999, '830.47739310', 4, '2.75787000', '830.47739310', '46530.51064628'] 
        
[1502942520000, '300.00000000', '300.00000000', '300.00000000', '300.00000000', '0.09930000', 1502942579999, '29.79000000', 2, '0.09930000', '29.79000000', '46530.51064628'] 
        
[1502942580000, '300.00000000', '300.00000000', '300.00000000', '300.00000000', '0.31389000', 1502942639999, '94.16700000', 3, '0.00000000', '0.00000000', '46530.51064628'] 
        
[1502942640000, '301.13000000', '301.13000000', '301.13000000', '301.13000000', '0.23202000', 1502942699999, '69.86818260', 1, '0.23202000', '69.86818260', '46530.96364628'] 
        
